# Loading data and preliminary analysis

In [ ]:
import pandas as pd
import nltk 
import numpy as np
import re

from nltk.stem import wordnet                                  
from sklearn.feature_extraction.text import CountVectorizer    
from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.model_selection import cross_val_score
from sklearn.tree import DecisionTreeClassifier  
from nltk import pos_tag                                       
from sklearn.metrics import pairwise_distances                
from nltk import word_tokenize                                 
from nltk.corpus import stopwords                              

In [ ]:
df = pd.read_csv("mentalhealth.csv", nrows = 20)
df.head()

,Question_ID,Questions,Answers
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl..."
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi..."
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...


In [ ]:
df.isnull().sum()

Question_ID    0
Questions      0
Answers        0
dtype: int64

# Clean data using NLTK






In [ ]:
nltk.download('punkt')   # The "punkt" package contains data necessary for tokenizing text into sentences and words.                  
   
s = 'tell me about your personality'
words = word_tokenize(s)                    
print(words)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


['tell', 'me', 'about', 'your', 'personality']


In [ ]:
nltk.download('wordnet')    #WordNet is a lexical database for the English language                    
lemma = wordnet.WordNetLemmatizer()         
lemma.lemmatize('absorbed', pos = 'v')        # lemmatize words

[nltk_data] Downloading package wordnet to /root/nltk_data...


'absorb'

In [ ]:
nltk.download('averaged_perceptron_tagger')      
pos_tag(nltk.word_tokenize(s),tagset = None)       # returns the parts of speech of every word

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


[('tell', 'VB'),
 ('me', 'PRP'),
 ('about', 'IN'),
 ('your', 'PRP$'),
 ('personality', 'NN')]

In [ ]:
nltk.download('stopwords')            

stop = stopwords.words('english')
print(stop)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
# function that performs text normalization steps and returns the lemmatized tokens as a sentence

def text_normalization(text):
    text = str(text).lower()                        # text to lower case
    spl_char_text = re.sub(r'[^ a-z]','',text)      # removing special characters
    tokens = nltk.word_tokenize(spl_char_text)      # word tokenizing
    lema = wordnet.WordNetLemmatizer()              # intializing lemmatization
    tags_list = pos_tag(tokens,tagset=None)         # parts of speech
    lema_words = []                                 # empty list 
    for token,pos_token in tags_list:               # lemmatize according to POS
        if pos_token.startswith('V'):               # Verb
            pos_val = 'v'
        elif pos_token.startswith('J'):             # Adjective
            pos_val = 'a'
        elif pos_token.startswith('R'):             # Adverb
            pos_val = 'r'
        else:
            pos_val = 'n'                           # Noun
        lema_token = lema.lemmatize(token,pos_val)

        if lema_token in stop: 
          lema_words.append(lema_token)             # appending the lemmatized token into a list
    
    return " ".join(lema_words) 

In [ ]:
text_normalization('telling you some stuffs about me')  

'you some about me'

In [ ]:
df['lemmatized_text'] = df['Questions'].apply(text_normalization)  
df.head(5)

,Question_ID,Questions,Answers,lemmatized_text
0,1590140,What does it mean to have a mental illness?,Mental illnesses are health conditions that di...,what do it to have a
1,2110618,Who does mental illness affect?,"Mental illness does can affect anyone, regardl...",who do
2,9434130,What are some of the warning signs of mental i...,Symptoms of mental health disorders vary depen...,what be some of the of
3,7657263,Can people with mental illness recover?,"When healing from mental illness, early identi...",can with
4,1619387,What should I do if I know someone who appears...,We encourage those with symptoms to talk to th...,what should i do if i who to have the of a


In [ ]:
cv = CountVectorizer()                                  # intializing the count vectorizer
X = cv.fit_transform(df['lemmatized_text']).toarray()

In [ ]:
# returns all the unique word from data 

features = cv.get_feature_names_out()
df_bow = pd.DataFrame(X, columns = features)
df_bow.head()

,about,after,and,be,before,between,can,do,for,have,...,or,should,some,the,this,to,what,where,who,with
0,0,0,0,0,0,0,0,1,0,1,...,0,0,0,0,0,1,1,0,0,0
1,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,1,0
2,0,0,0,1,0,0,0,0,0,0,...,0,0,1,1,0,0,1,0,0,0
3,0,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,1,0,1,...,0,1,0,1,0,1,1,0,1,0


In [ ]:
Question = 'What treatment options are available'                           

In [ ]:
Question_lemma = text_normalization(Question) 
                             
Question_bow = cv.transform([Question_lemma]).toarray()   
             

# Cosine similarity

In [ ]:
# cosine similarity for the above question we considered.

cosine_value = 1- pairwise_distances(df_bow, Question_bow, metric = 'cosine' )
(cosine_value)

array([[0.31622777],
       [0.        ],
       [0.5       ],
       [0.        ],
       [0.23570226],
       [0.        ],
       [1.        ],
       [0.31622777],
       [0.70710678],
       [0.        ],
       [0.31622777],
       [0.        ],
       [0.40824829],
       [0.25      ],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.70710678],
       [0.        ],
       [0.        ]])

In [ ]:
df['similarity_bow'] = cosine_value                                         

In [ ]:
simiscores = pd.DataFrame(df, columns=['Answers','similarity_bow'])         # taking similarity value of responses for the question we took
simiscores

,Answers,similarity_bow
0,Mental illnesses are health conditions that di...,0.316228
1,"Mental illness does can affect anyone, regardl...",0.000000
2,Symptoms of mental health disorders vary depen...,0.500000
3,"When healing from mental illness, early identi...",0.000000
4,We encourage those with symptoms to talk to th...,0.235702
5,Feeling comfortable with the professional you ...,0.000000
6,Different treatment options are available for ...,1.000000
7,It is important to be as involved and engaged ...,0.316228
8,There are many types of mental health professi...,0.707107
9,Feeling comfortable with the professional you ...,0.000000


In [ ]:
simscoresDescending = simiscores.sort_values(by = 'similarity_bow', ascending=False)          # sorting the values
simscoresDescending.head()

,Answers,similarity_bow
6,Different treatment options are available for ...,1.000000
17,There are many types of mental health professi...,0.707107
8,There are many types of mental health professi...,0.707107
2,Symptoms of mental health disorders vary depen...,0.500000
12,The best source of information regarding medic...,0.408248


In [ ]:
threshold = 0.1                                                                         # considering the value of smiliarity to be greater than 0.1
df_threshold = simscoresDescending[simscoresDescending['similarity_bow'] > threshold] 
df_threshold

,Answers,similarity_bow
6,Different treatment options are available for ...,1.000000
17,There are many types of mental health professi...,0.707107
8,There are many types of mental health professi...,0.707107
2,Symptoms of mental health disorders vary depen...,0.500000
12,The best source of information regarding medic...,0.408248
0,Mental illnesses are health conditions that di...,0.316228
7,It is important to be as involved and engaged ...,0.316228
10,It is important to continue involvement in the...,0.316228
13,Create a plan for switching to a different tre...,0.250000
4,We encourage those with symptoms to talk to th...,0.235702


In [ ]:
index_value = cosine_value.argmax()         # index number of highest value
index_value

6

In [ ]:
# Question = 'What treatment options are available'                           
df['Answers'].loc[index_value]              # The text at the above index becomes the response for the question

'Different treatment options are available for individuals with mental illness.'

# Tf-Idf

In [ ]:
Question1 = 'What treatment options are available'

In [ ]:
# using tf-idf

tfidf = TfidfVectorizer()                                             # intializing tf-id 
x_tfidf = tfidf.fit_transform(df['lemmatized_text']).toarray()        # transforming the data into array

In [ ]:
Question_lemma1 = text_normalization(Question1)
Question_tfidf = tfidf.transform([Question_lemma1]).toarray()         # applying tf-idf

In [ ]:
# returns all the unique word from data with a score of that word

df_tfidf = pd.DataFrame(x_tfidf,columns = tfidf.get_feature_names_out()) 
df_tfidf.head()

,about,after,and,be,before,between,can,do,for,have,...,or,should,some,the,this,to,what,where,who,with
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.392029,0.0,0.550307,...,0.0,0.000000,0.000000,0.000000,0.0,0.367085,0.325401,0.0,0.000000,0.000000
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.580211,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.814466,0.000000
2,0.0,0.0,0.0,0.321859,0.0,0.0,0.000000,0.000000,0.0,0.000000,...,0.0,0.000000,0.478821,0.347908,0.0,0.000000,0.248876,0.0,0.000000,0.000000
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.440977,0.000000,0.0,0.000000,...,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.897519
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.282658,0.0,0.396779,...,0.0,0.396779,0.000000,0.327977,0.0,0.264673,0.234618,0.0,0.396779,0.000000


In [ ]:
cos = 1-pairwise_distances(df_tfidf,Question_tfidf,metric='cosine')                     # applying cosine similarity
cos

array([[0.19904882],
       [0.        ],
       [0.40685646],
       [0.        ],
       [0.14351684],
       [0.        ],
       [1.        ],
       [0.20934186],
       [0.56647821],
       [0.        ],
       [0.20934186],
       [0.        ],
       [0.22245129],
       [0.22913146],
       [0.        ],
       [0.        ],
       [0.        ],
       [0.6372619 ],
       [0.        ],
       [0.        ]])

In [ ]:
df['similarity_tfidf'] = cos                                                    # creating a new column 
df_simi_tfidf = pd.DataFrame(df, columns=['Answers','similarity_tfidf'])        # taking similarity value of responses for the question we took
df_simi_tfidf

,Answers,similarity_tfidf
0,Mental illnesses are health conditions that di...,0.199049
1,"Mental illness does can affect anyone, regardl...",0.000000
2,Symptoms of mental health disorders vary depen...,0.406856
3,"When healing from mental illness, early identi...",0.000000
4,We encourage those with symptoms to talk to th...,0.143517
5,Feeling comfortable with the professional you ...,0.000000
6,Different treatment options are available for ...,1.000000
7,It is important to be as involved and engaged ...,0.209342
8,There are many types of mental health professi...,0.566478
9,Feeling comfortable with the professional you ...,0.000000


In [ ]:
df_simi_tfidf_sort = df_simi_tfidf.sort_values(by='similarity_tfidf', ascending=False)            # sorting the values
df_simi_tfidf_sort.head(10)

,Answers,similarity_tfidf
6,Different treatment options are available for ...,1.000000
17,There are many types of mental health professi...,0.637262
8,There are many types of mental health professi...,0.566478
2,Symptoms of mental health disorders vary depen...,0.406856
13,Create a plan for switching to a different tre...,0.229131
12,The best source of information regarding medic...,0.222451
10,It is important to continue involvement in the...,0.209342
7,It is important to be as involved and engaged ...,0.209342
0,Mental illnesses are health conditions that di...,0.199049
4,We encourage those with symptoms to talk to th...,0.143517


In [ ]:
threshold = 0.1                                                                                   # considering the value of smiliarity to be greater than 0.1
df_threshold = df_simi_tfidf_sort[df_simi_tfidf_sort['similarity_tfidf'] > threshold] 
df_threshold

,Answers,similarity_tfidf
6,Different treatment options are available for ...,1.000000
17,There are many types of mental health professi...,0.637262
8,There are many types of mental health professi...,0.566478
2,Symptoms of mental health disorders vary depen...,0.406856
13,Create a plan for switching to a different tre...,0.229131
12,The best source of information regarding medic...,0.222451
10,It is important to continue involvement in the...,0.209342
7,It is important to be as involved and engaged ...,0.209342
0,Mental illnesses are health conditions that di...,0.199049
4,We encourage those with symptoms to talk to th...,0.143517


In [ ]:
index_value1 = cos.argmax()                                                   # returns the index number of highest value
index_value1

6

In [ ]:
df['Answers'].loc[index_value1]                                               # returns the text at that index

'Different treatment options are available for individuals with mental illness.'

# Testing chatbot

In [ ]:
# defining a function that returns response to query using bow

def chat_bow(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    bow = cv.transform([lemma]).toarray() # applying bow
    cosine_value = 1- pairwise_distances(df_bow,bow, metric = 'cosine' )
    index_value = cosine_value.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [ ]:
chat_bow('can you prevent mental health problems')

'When healing from mental illness, early identification and treatment are of vital importance. '

In [ ]:
chat_bow('what is mental health')

'Different treatment options are available for individuals with mental illness.'

In [ ]:
chat_bow('are there cures for mental health problems')

'Different treatment options are available for individuals with mental illness.'

In [ ]:
chat_bow('how do I know if i am unwell')

'Create a plan for switching to a different treatment that will be a better fit.'

In [ ]:
chat_bow('what do you mean by mental health')

"Mental illnesses are health conditions that disrupt a person's thoughts, emotions, relationships, and daily functioning."

In [ ]:
# defining a function that returns response to query using tf-idf

def chat_tfidf(text):
    lemma = text_normalization(text) # calling the function to perform text normalization
    tf = tfidf.transform([lemma]).toarray() # applying tf-idf
    cos = 1-pairwise_distances(df_tfidf,tf,metric='cosine') # applying cosine similarity
    index_value = cos.argmax() # getting index value 
    return df['Answers'].loc[index_value]

In [ ]:
chat_tfidf('what is mental health')

'Different treatment options are available for individuals with mental illness.'

In [ ]:
chat_tfidf('what is vaping')

'Different treatment options are available for individuals with mental illness.'

In [ ]:
chat_tfidf('how do i see a counsellor')

'Visit Healthfinder.gov to learn more.'

In [ ]:
chat_tfidf('how to find a support group')

'Visit Healthfinder.gov to learn more.'